### Script
- **Input:** FAQ generated in 2b and listing entities generated in 2d.
- **Output:** auto-generated nlu.yml for RASA solution.
    
- Approx 300K intents

In [85]:
root = './RASA_ConceptNet5/data/'
processed = './Data/processing/Processed_Airbnb/'

### Import

In [86]:
import pandas as pd 

## Generate nlu.yml

### Gather FAQs

In [87]:
FAQ1 = pd.read_csv(processed+'FAQ_1.json')
FAQ2 = pd.read_csv(processed+'FAQ_2.json')
FAQ3 = pd.read_csv(processed+'FAQ_3.json')


FAQ =  pd.concat([FAQ1, FAQ2, FAQ3], axis=0)

FAQ.head(1)

,question,answer,listing_id,context
0,Where did I find the place from?,central station,2818.0,I didnt have any trouble finding the place fro...


### Add intent type

In [88]:
FAQ['intent'] = 'affirm_FAQ'
FAQ['type'] = 'intent'

In [89]:
FAQ.head(5)

,question,answer,listing_id,context,intent,type
0,Where did I find the place from?,central station,2818.0,I didnt have any trouble finding the place fro...,affirm_FAQ,intent
1,Who is really cool?,daniel,2818.0,Daniel is really cool.,affirm_FAQ,intent
2,What was in the room for you to use?,maps,2818.0,He had maps and a lonely planet guide book in ...,affirm_FAQ,intent
3,What is the best book you could ever need?,guidebooks,2818.0,"His place is extremely clean, and he provides ...",affirm_FAQ,intent
4,What is the best thing about the place?,comfy bed,2818.0,"His place is extremely clean, and he provides ...",affirm_FAQ,intent


### Append entity and entity value to the intent.

In [90]:
#Remove ? from the question 
#Append entity value and entity at the end and append '?' => [answer](wordid)?

FAQ['question'] = FAQ['question'].str.replace(r'?', ' ',regex=True)
FAQ.head(5)

,question,answer,listing_id,context,intent,type
0,Where did I find the place from,central station,2818.0,I didnt have any trouble finding the place fro...,affirm_FAQ,intent
1,Who is really cool,daniel,2818.0,Daniel is really cool.,affirm_FAQ,intent
2,What was in the room for you to use,maps,2818.0,He had maps and a lonely planet guide book in ...,affirm_FAQ,intent
3,What is the best book you could ever need,guidebooks,2818.0,"His place is extremely clean, and he provides ...",affirm_FAQ,intent
4,What is the best thing about the place,comfy bed,2818.0,"His place is extremely clean, and he provides ...",affirm_FAQ,intent


In [91]:
# FAQ['question'] = FAQ['question'].astype(str) +'['+FAQ['answer'].astype(str)+'](wordid)?'
FAQ['question'] = FAQ['question'].astype(str)

In [92]:
FAQ.head(5)

,question,answer,listing_id,context,intent,type
0,Where did I find the place from,central station,2818.0,I didnt have any trouble finding the place fro...,affirm_FAQ,intent
1,Who is really cool,daniel,2818.0,Daniel is really cool.,affirm_FAQ,intent
2,What was in the room for you to use,maps,2818.0,He had maps and a lonely planet guide book in ...,affirm_FAQ,intent
3,What is the best book you could ever need,guidebooks,2818.0,"His place is extremely clean, and he provides ...",affirm_FAQ,intent
4,What is the best thing about the place,comfy bed,2818.0,"His place is extremely clean, and he provides ...",affirm_FAQ,intent


### Auto-generate NLU intents

In [93]:
nlus = FAQ.groupby('intent')['question'].apply(list).to_dict()

In [94]:
answer = FAQ["answer"].unique()
nlu=[]
for val in answer:
    questionString1="find me close matches for a place near "
    questionString2="get me recommendations with features close to"
    questionString3="I am looking for a place with facilities similar to "
    questionString4="get me recommendations with facilities similar to "
    questionString5="find similar recommendations like a "
    questionString6="find places with amenities similar to "
    questionString7="find places suitable for "
     

    n = val
    nlu.append(questionString1+"["+str(n)+"](wordid)")
    nlu.append(questionString2+"["+str(n)+"](wordid)")
    nlu.append(questionString3+"["+str(n)+"](wordid)")
    nlu.append(questionString4+"["+str(n)+"](wordid)")
    nlu.append(questionString5+"["+str(n)+"](wordid)")
    nlu.append(questionString6+"["+str(n)+"](wordid)")

In [95]:
#;TYPE;INTENT;VALUE
f = open(root+"nlu.yml","a")

for intent in nlus:
    f.write("\n- intent: {}\n".format(intent))
    f.write("  examples: |\n")
    for value in nlus[intent]:
        f.write("    - {}\n".format(value))

f.write("\n- intent: {}\n".format("affirm_neo4j_conceptNet5_review"))
f.write("  examples: |\n")
for value in nlu:
    f.write("    - {}\n".format(value.lower()))
f.close()

### Generate listings nlu

In [96]:
import csv 

listing_reviews = pd.read_csv(processed+'listings_text_processed.csv')

listing_reviews = listing_reviews[['neighbourhood','neighbourhood_cleansed','property_type'
                                  ,'room_type','accommodates','bathrooms','bathrooms_text'
                                  ,'bedrooms','beds','amenities','price']]


neighborhood =  listing_reviews['neighbourhood'].unique()
loc =  listing_reviews['neighbourhood'].unique()
accomodates = listing_reviews['accommodates'].unique()
bathrooms = listing_reviews['bathrooms'].unique()
bedrooms = listing_reviews['bedrooms'].unique()
beds = listing_reviews['beds'].unique()
price = listing_reviews['price'].unique()
bathrooms_text = listing_reviews['bathrooms_text'].unique()
property_type = listing_reviews['property_type'].unique()
room_type = listing_reviews['room_type'].unique()
amenities = listing_reviews['amenities'].unique()

nlu = []
for val in loc:
    questionString1="find me a place near "
    questionString2="get me recommendations with "
    questionString3="I am looking for a place with facilities like "
    questionString4="get me recommendations with facilities to " 

    if ',' in val:
        neigh = val.split(',')
        for n in neigh:
            nlu.append(questionString1+"["+n.strip('\"').strip()+"](wordid)")
            nlu.append(questionString2+"["+n.strip('\"').strip()+"](wordid)")
            nlu.append(questionString3+"["+n.strip('\"').strip()+"](wordid)")
            nlu.append(questionString4+"["+n.strip('\"').strip()+"](wordid)")
    else:
        n = val
        nlu.append(questionString1+"["+n.strip('\"').strip()+"](wordid)")
        nlu.append(questionString2+"["+n.strip('\"').strip()+"](wordid)")
        nlu.append(questionString3+"["+n.strip('\"').strip()+"](wordid)")
        nlu.append(questionString4+"["+n.strip('\"').strip()+"](wordid)")

for val in neighborhood:
    questionString1="find me a place in the neighborhood of "
    questionString2="find me a place in "

    if ',' in val:
        neigh = val.split(',')
        for n in neigh:
            nlu.append(questionString1+"["+n.strip('\"').strip()+"](wordid)")
            nlu.append(questionString2+"["+n.strip('\"').strip()+"](wordid)")
    else:
        n = val
        nlu.append(questionString1+"["+n.strip('\"').strip()+"](wordid)")
        nlu.append(questionString2+"["+n.strip('\"').strip()+"](wordid)")

for val in accomodates:
    questionString1="find me a place that can accomodate "
    questionString2="find me a place for "
    
    n = str(val)
    nlu.append(questionString1+"["+n.strip('\"').strip()+"](wordid)")
    nlu.append(questionString2+"["+n.strip('\"').strip()+"](wordid)")

for val in bathrooms:
    questionString1="find me a place with "
    questionString2="looking for a place with "

    n = str(val)
    nlu.append(questionString1+"["+n.strip('\"').strip()+"](wordid) bathrooms")
    nlu.append(questionString2+"["+n.strip('\"').strip()+"](wordid) bathrooms")


for val in bedrooms:
    questionString1="find me a place with "
    questionString2="looking for a place with "

    n = str(val)
    nlu.append(questionString1+"["+n.strip('\"').strip()+"](wordid) bedrooms")
    nlu.append(questionString2+"["+n.strip('\"').strip()+"](wordid) bedrooms")

for val in beds:
    questionString1="find me a place with "
    questionString2="looking for a place with "

    n = str(val)
    nlu.append(questionString1+"["+n.strip('\"').strip()+"](wordid) beds")
    nlu.append(questionString2+"["+n.strip('\"').strip()+"](wordid) beds")

for val in price:
    questionString1="find me a place in the range of "
    questionString2="looking for a place within the price "

    n = val
    nlu.append(questionString1+"["+n.strip('\"').strip()+"](wordid)")
    nlu.append(questionString2+"["+n.strip('\"').strip()+"](wordid)")

for val in bathrooms_text:
    questionString1="find me a place with bath that is "
    questionString2="looking for a place with bath features like "

    n = str(val)
    nlu.append(questionString1+"["+n.strip('\"').strip()+"](wordid)")
    nlu.append(questionString2+"["+n.strip('\"').strip()+"](wordid)")

for val in amenities:
    questionString1="find me a place with amenities like "
    questionString2="I am looking for a place with amenities like "
    val = val.strip('[').strip(']')
    if ',' in val:
        neigh = val.split(',')
        for n in neigh:
            nlu.append(questionString1+"["+n.replace('"', '').strip()+"](wordid)")
            nlu.append(questionString2+"["+n.replace('"', '').strip()+"](wordid)")
    else:
        n = val
        nlu.append(questionString1+"["+n.strip('\"').strip()+"](wordid)")
        nlu.append(questionString2+"["+n.strip('\"').strip()+"](wordid)")

for val in property_type:
    questionString1="find me a "
    questionString2="looking for a "

    if ',' in val:
        neigh = val.split(',')
        for n in neigh:
            nlu.append(questionString1+"["+n.strip('\"').strip()+"](wordid) property")
            nlu.append(questionString2+"["+n.strip('\"').strip()+"](wordid) property")
    else:
        n = val
        nlu.append(questionString1+"["+n.strip('\"').strip()+"](wordid) property")
        nlu.append(questionString2+"["+n.strip('\"').strip()+"](wordid) property")


for val in room_type:
    questionString1="find me a "
    questionString2="looking for a"

    if ',' in val:
        neigh = val.split(',')
        for n in neigh:
            nlu.append(questionString1+"["+n.strip('\"').strip()+"](wordid) room")
            nlu.append(questionString2+"["+n.strip('\"').strip()+"](wordid) room")
    else:
        n = val
        nlu.append(questionString1+"["+n.strip('\"').strip()+"](wordid) room")
        nlu.append(questionString2+"["+n.strip('\"').strip()+"](wordid) room")



In [ ]:
#;TYPE;INTENT;VALUE
f = open(root+"nlu.yml","a")

f.write("\n- intent: {}\n".format("affirm_neo4j_conceptNet5_listing"))
f.write("  examples: |\n")
for value in nlu:
    f.write("    - {}\n".format(value.lower()))
f.close()